In [77]:
import pandas as pd
import EDA as eda
from eda_helper import rows_to_del_with_index
from eda_helper import createExcel
import re
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
df1_rec=eda.df1 #creating copy of dataframe for recommendation notebook

In [3]:
#df1_rec.info() #52262 entries - datetime indexed

In [4]:
df1_rec=df1_rec.reset_index() #changing index to int

In [5]:
df1_rec.head()

,date,uid,category,categoryid,simple_description,amountnum,city,state,zip_code,address,latitude,longitude,accountid
0,2020-08-20,WdtUjyA3mghZ6TrphronQpCVrI23,Entertainment,7.0,sirius xm,20.63,NaN,None,NaN,None,NaN,NaN,10038937.0
1,2020-08-20,WdtUjyA3mghZ6TrphronQpCVrI23,Restaurants/Dining,22.0,*pending debit aug20 p 2818at buona glen ellyn...,20.69,Glen Ellyn,None,None,None,NaN,NaN,10038937.0
2,2020-08-20,WdtUjyA3mghZ6TrphronQpCVrI23,Groceries,10.0,the beer cellar,28.51,Glen Ellyn,None,None,None,NaN,NaN,10038937.0
3,2020-08-20,WdtUjyA3mghZ6TrphronQpCVrI23,Entertainment,7.0,disney plus,6.99,NaN,CA,NaN,None,NaN,NaN,10038937.0
4,2020-08-20,WdtUjyA3mghZ6TrphronQpCVrI23,Restaurants/Dining,22.0,p #2818christ cafe 95 oak lawn il x0232,1.09,Oak Lawn,IL,None,None,NaN,NaN,10038937.0


In [6]:
rowstodel=df1_rec[df1_rec.simple_description.str.contains('shudder.com')].index #shudder is a movie website. Not aresturant/dinning.
rows_to_del_with_index(df1_rec,rowstodel)

In [7]:
rowstodel=df1_rec[df1_rec.simple_description.str.contains('withdrawal')].index #withdrawls should have been in "withdrawal" category
rows_to_del_with_index(df1_rec,rowstodel)

In [8]:
rowstodel=df1_rec[df1_rec.simple_description.str.contains('tdxx')].index #All these trans are by one user who used SamPay to pay for some fastfood in NC while on "copperline tour". no resturant info is available.
rows_to_del_with_index(df1_rec,rowstodel) 

In [9]:
rowstodel=df1_rec[df1_rec.simple_description.str.contains('wpc@tabernacle6920')].index #Some concert food? deleting for now. Only 3 records 
rows_to_del_with_index(df1_rec,rowstodel)

In [10]:
rowstodel=df1_rec[(df1_rec.simple_description.str.contains('transfer')) & (df1_rec.categoryid==22.0)].index #transfer of some sort. Not food related
rows_to_del_with_index(df1_rec,rowstodel)

In [11]:
rowstodel=df1_rec[df1_rec.simple_description.str.contains('authorization')].index #one row with start word "authorization"
rows_to_del_with_index(df1_rec,rowstodel)

In [12]:
rowstodel=df1_rec[df1_rec.simple_description.str.contains('bethel university') & (df1_rec.categoryid==22.0)].index #65 rows with description as "bethel university"??? what is it doing in this category?
rows_to_del_with_index(df1_rec,rowstodel)

In [13]:
rowstodel=df1_rec[df1_rec.simple_description.str.contains('at&t') & (df1_rec.categoryid==22.0)].index
rows_to_del_with_index(df1_rec,rowstodel)

In [14]:
df1_rec[df1_rec.simple_description.str.contains(r"^cafe")].groupby('simple_description').count()

#Question for FinGoal: What is cafe *******. It has most record here and it is not giving any specific info.
#Keep them as is or?

,date,uid,category,categoryid,amountnum,city,state,zip_code,address,latitude,longitude,accountid
simple_description,,,,,,,,,,,,
cafe *******,35,35,35,35,35,33,33,33,19,19,19,35
cafe agora,1,1,1,1,1,0,0,0,0,0,0,1
cafe beirut,1,1,1,1,1,0,0,0,0,0,0,1
cafe brioche,5,5,5,5,5,4,5,0,0,0,0,5
cafe du monde - will kenner la 07/15,1,1,1,1,1,1,1,0,0,0,0,1
cafe frederica,1,1,1,1,1,0,0,0,0,0,0,1
cafe hon,2,2,2,2,2,2,2,2,2,2,2,2
cafe hunan,1,1,1,1,1,0,0,0,0,0,0,1
cafe jordano,1,1,1,1,1,0,0,0,0,0,0,1


In [15]:
df1_rec[df1_rec.simple_description.str.contains(r"^brewing")].groupby('simple_description').count()

### Question for FinGoal: What is this "brewing *******et" and it's other versions? keep them as is or?

,date,uid,category,categoryid,amountnum,city,state,zip_code,address,latitude,longitude,accountid
simple_description,,,,,,,,,,,,
brewing *******et,111,111,111,111,111,104,47,104,40,40,40,111
brewing *******et 13th str,1,1,1,1,1,0,0,0,0,0,0,1
brewing *******et coffee,4,4,4,4,4,4,4,4,4,4,4,4
brewing *******et lafayett,4,4,4,4,4,0,0,0,0,0,0,4
brewing *******et longmont,20,20,20,20,20,0,0,0,0,0,0,20


In [16]:
df1_rec.reset_index(inplace=True, drop=True)

In [17]:
#df1_rec.info() #51684 entries

In [18]:
#df1_rec[df1_rec.simple_description.str.contains('christ cafe')]

#christ cafe same as christi cafe

In [19]:
df1_rec.head()

,date,uid,category,categoryid,simple_description,amountnum,city,state,zip_code,address,latitude,longitude,accountid
0,2020-08-20,WdtUjyA3mghZ6TrphronQpCVrI23,Entertainment,7.0,sirius xm,20.63,NaN,None,NaN,None,NaN,NaN,10038937.0
1,2020-08-20,WdtUjyA3mghZ6TrphronQpCVrI23,Restaurants/Dining,22.0,*pending debit aug20 p 2818at buona glen ellyn...,20.69,Glen Ellyn,None,None,None,NaN,NaN,10038937.0
2,2020-08-20,WdtUjyA3mghZ6TrphronQpCVrI23,Groceries,10.0,the beer cellar,28.51,Glen Ellyn,None,None,None,NaN,NaN,10038937.0
3,2020-08-20,WdtUjyA3mghZ6TrphronQpCVrI23,Entertainment,7.0,disney plus,6.99,NaN,CA,NaN,None,NaN,NaN,10038937.0
4,2020-08-20,WdtUjyA3mghZ6TrphronQpCVrI23,Restaurants/Dining,22.0,p #2818christ cafe 95 oak lawn il x0232,1.09,Oak Lawn,IL,None,None,NaN,NaN,10038937.0


##### From David's email this morning "The thought is to be recommending merchants. Eg “Folks like you are enjoying TJ Maxx.” to someone who has similar transactions to those who go to TJ Maxx, but isnt currently a customer there."

#### To  implement above scenario, I need to first figure out the merchants in the data. Simple_description has text and I can extract the "merchant" information from it but since it's all different length text with different start and end word etc., I will focus on extracting merchants from one category at a time. 

## Extraction of Merchant Information From Simple_Description

In [20]:
### Finding how many records exists per category. 

In [21]:
df1_cat_catid_simple=df1_rec.groupby(['category','categoryid'])['simple_description'].count().to_frame()

In [22]:
df1_cat_catid_simple.sort_values('simple_description', ascending=False)

,,simple_description
category,categoryid,
Restaurants/Dining,22.0,8419
General Merchandise,44.0,6368
Shops,201.0,6230
Service,202.0,5137
Groceries,10.0,4718
Travel,23.0,3702
Food and Drink,203.0,2877
Clothing/Shoes,5.0,1877
Entertainment,7.0,1754


### From above dataframe, we can see the largest records are in Resturants/Dining category (categoryid=22)

### Creating DF of all the records for categoryid=22 only to focus on extracting the merchants 

In [23]:
#Resturants/Dinning
df1_rec[df1_rec.categoryid==22.0]['simple_description'].nunique() #2514 unique texts(names) in simple_description

2514

In [24]:
#resturant_list=df1_rec[df1_rec.categoryid==22.0]['simple_description'].unique().tolist()

In [25]:
df1_rec_resturants=df1_rec[df1_rec.categoryid==22.0] # All records with categoryid=22.0 (resturants/dinning)
df1_rec_resturants.reset_index(inplace=True,drop=True) #8419 records
df1_rec_resturants.info() #8419 records

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8419 entries, 0 to 8418
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   date                8419 non-null   datetime64[ns]
 1   uid                 8419 non-null   object        
 2   category            8419 non-null   object        
 3   categoryid          8419 non-null   float64       
 4   simple_description  8419 non-null   object        
 5   amountnum           8419 non-null   float64       
 6   city                4589 non-null   object        
 7   state               4485 non-null   object        
 8   zip_code            3039 non-null   object        
 9   address             1801 non-null   object        
 10  latitude            1815 non-null   float64       
 11  longitude           1815 non-null   float64       
 12  accountid           8419 non-null   float64       
dtypes: datetime64[ns](1), float64(5), object(7)
memo

In [26]:
df1_rec_resturants.head()

,date,uid,category,categoryid,simple_description,amountnum,city,state,zip_code,address,latitude,longitude,accountid
0,2020-08-20,WdtUjyA3mghZ6TrphronQpCVrI23,Restaurants/Dining,22.0,*pending debit aug20 p 2818at buona glen ellyn...,20.69,Glen Ellyn,None,None,None,NaN,NaN,10038937.0
1,2020-08-20,WdtUjyA3mghZ6TrphronQpCVrI23,Restaurants/Dining,22.0,p #2818christ cafe 95 oak lawn il x0232,1.09,Oak Lawn,IL,None,None,NaN,NaN,10038937.0
2,2020-08-20,A4Kyy9PFU7QLr08iECIlJOMI0zx2,Restaurants/Dining,22.0,chick-fil-a,12.22,Littleton,CO,None,None,NaN,NaN,10092987.0
3,2020-08-20,A4Kyy9PFU7QLr08iECIlJOMI0zx2,Restaurants/Dining,22.0,the original pancake house,18.00,Greenwood Village,CO,None,None,NaN,NaN,10092987.0
4,2020-08-20,kSCSRuyQqWOxp0kPlXwwF0dmhQ02,Restaurants/Dining,22.0,sq *world's end brewin,63.49,NaN,None,NaN,None,NaN,NaN,10293258.0


In [27]:
#createExcel('merchant.xlsx',pd.DataFrame(df1_rec_resturants['simple_description'].values))

### Scan through the texts in column 'simple_description' and by observing different starting strings, extract the name of the merchants from the column and add to resturant list (rest_list)

In [28]:
rest_list=[]
for merch in df1_rec_resturants.simple_description:
    if (('zaza italian' in merch) or ('zazaitalian' in merch)):
        rest_list.append('zaza italian')
    elif (('five & hoe' in merch) or ('*five & hoe') in merch):
        rest_list.append('five & hoek coffee')
    elif (('a & w' in merch)):
        rest_list.append('a & w')
    elif (('a mano' in merch)):
        rest_list.append('a mano')
    elif (('fridas mexican' in merch)):
        rest_list.append('fridas mexican')
    elif (('zucca bar' in merch)):
        rest_list.append('zucca bar & pizzeria')
    elif (('yonah mountain' in merch)):
        rest_list.append('yonah mountain')
    elif ('not just coffe' in merch):
        rest_list.append('not just coffee')
    elif (('ziggy' in merch) or ('ziggi' in merch)):
        rest_list.append("ziggi's coffee")
    elif('winot coffee' in merch):
        rest_list.append('winot coffee')
    elif ('christ cafe' in merch):
        rest_list.append('christi cafe') #christ cafe and christi cafe are the same in lawn oak, IL. 
    elif('jack & winn' in merch):
        rest_list.append('jack & winn')
    elif('taj mahal' in merch):
        rest_list.append('taj mahal')
    elif ('two scoops' in merch):
        rest_list.append('two scoops plaza')
    elif('chamberlain coff' in merch):
        rest_list.append('chamberlain coffee')
    elif('warren' in merch):
        rest_list.append('warrens ale house')
    elif(('wing shack' in merch) or ('wing shack*' in merch)):
        rest_list.append('wing shack')
    elif('wild frontiers' in merch):
        rest_list.append('wild frontiers')
    elif('wahlburgers' in merch):
        rest_list.append('wahlburgers')
    elif('viva chicken' in merch):
        rest_list.append('viva chicken')
    elif('vail brewing' in merch):
        rest_list.append('vail brewing')
    elif('two hound red' in merch):
        rest_list.append('two hound red')
    elif('la vita bella' in merch):
        rest_list.append('la vita bella')
    elif('mikerphone' in merch):
        rest_list.append('mikerphone brewery')
    elif(merch.startswith('tst* ')):
        rest_list.append(merch[len('tst* '):])
    elif('in *the sing' in merch):
        rest_list.append('in *the singing cook')
    elif('dk diner' in merch):
        rest_list.append('dk diner')
    elif('dbt the broadmoor fo' in merch):
        rest_list.append('the broadmoor, CO Springs')
    elif ('dbt tst* attic bar' in merch):
        rest_list.append('attic bar')
    elif('dbt twisted pine' in merch):
        rest_list.append('twisted pine')
    elif('dbt tst* walnut cafe' in merch):
        rest_list.append('walnut cafe, Boulder CO')
    elif('dbt tundra restauran' in merch):
        rest_list.append('thundra resturant')
    elif('dbt rayback collecti' in merch):
        rest_list.append('rayback collection')
    elif('trustworthy' in merch):
        rest_list.append('trustworthy brewin, Las Vegas NV')
    elif('12degree' in merch):
        rest_list.append('12degree brewin')
    elif('card' in merch):
        rest_list.append(re.sub("card \d+",' ',merch))
    elif('card' in merch):
        rest_list.append(re.sub("card\d+",' ',merch))
    elif('benny' in merch):
        rest_list.append("bennys tacos")
    elif('big daddy bagel' in merch):
        rest_list.append('big daddy bagels, Lafayette CO')
    elif('boulder creek' in merch):
        rest_list.append('boulder creek coffee')
    elif('bitesquad deli' in merch):
        rest_list.append('bitesquad deli')
        
    
    elif('ara u of m caribou cminneapolis' in merch):
        rest_list.append('Caribou Coffee University Of Minnesota')
        

    elif ((merch[0:8]=='*pending')):
        rest_list.append(merch[30:])
    elif (merch[0:7]=='pending'):
        rest_list.append(merch[23:])
    elif (merch[0:2]=='p '):
        rest_list.append(merch[11:25])
    elif (merch[0:2]=='pp'):
        rest_list.append(merch[3:])
    elif (merch[0:2]=='a '):
        rest_list.append(merch[7:])
    elif (merch[0:8]=='visa pp*'):
        rest_list.append(merch[8:])
    elif (merch[0:7]=='visa sq'):
        rest_list.append(merch[9:])
    elif (merch[0:4]=='visa'):
        rest_list.append(merch[5:26])
    elif (merch[0:2]=='sq'):
        rest_list.append(merch[4:])
    elif (merch[0:2]=='sp'):
        rest_list.append(merch[5:])
    else:
        rest_list.append(merch)
        
#rest_list #visa visa pp* 'sq, visa jefes tacos & tequila 'yonah mountain fridas mexican zucca bar ziggy tandur indi, not just coffe, winot coffee, christ cafe
sorted(rest_list)

df_sorted_list = pd.DataFrame (sorted(rest_list),columns=['After_Filtering']) #8419
df_sorted_list
#createExcel('merchant_filtered.xlsx',df_sorted_list)

,After_Filtering
0,
1,and stable
2,bray gour
3,brewing compa
4,brewing compa
...,...
8414,zucca bar & pizzeria
8415,zucca bar & pizzeria
8416,zucca bar & pizzeria
8417,zucca bar & pizzeria


In [29]:
df1_rec_resturants.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8419 entries, 0 to 8418
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   date                8419 non-null   datetime64[ns]
 1   uid                 8419 non-null   object        
 2   category            8419 non-null   object        
 3   categoryid          8419 non-null   float64       
 4   simple_description  8419 non-null   object        
 5   amountnum           8419 non-null   float64       
 6   city                4589 non-null   object        
 7   state               4485 non-null   object        
 8   zip_code            3039 non-null   object        
 9   address             1801 non-null   object        
 10  latitude            1815 non-null   float64       
 11  longitude           1815 non-null   float64       
 12  accountid           8419 non-null   float64       
dtypes: datetime64[ns](1), float64(5), object(7)
memo

In [30]:
#Added column "Merchant" in the dataframe df1_rec_resturants
df1_rec_resturants.loc[:,'merchant'] = pd.Series(rest_list, index=df1_rec_resturants.index)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


### Columns to be used for filtering once we have similarity scores. 

In [31]:
## Question for Fingoal: Need input if these filters are OK ?

In [32]:
count_per_merch=df1_rec_resturants.groupby(['merchant'])['merchant'].count()
count_per_merch.sort_values(ascending=False) #Over thousand resturants have only 1 entry in the dataframe.

merchant
starbucks                     398
chipotle mexican grill        255
dunkin' donuts                255
chick-fil-a                   245
firehouse subs                221
                             ... 
offerdahl cafe t1 fll           1
old chicago                     1
old irving brewing chicago      1
olde hickory tap room           1
                                1
Name: merchant, Length: 2225, dtype: int64

In [33]:
meanamt_per_merch=df1_rec_resturants.groupby(['merchant'])['amountnum'].mean()
meanamt_per_merch.sort_values() # (shape: 2225 x 1)

merchant
lim*restore san francisco ca card0591                   0.880000
douglas county school district nutrition services       1.000000
double barrel                                           1.000000
curiositystr silver spring md card8563                  1.000000
cloosiv                                                 1.080000
                                                        ...     
acre                                                  806.660000
rolando's                                            1294.810000
flying b bar ranch llc strasburg us                  1405.277778
blind barber                                         2302.500000
parlay                                               3111.560000
Name: amountnum, Length: 2225, dtype: float64

In [34]:
df1_rec_merch_count_mtx=df1_rec_resturants.pivot_table(index='uid',columns='merchant', values='amountnum', aggfunc='count', fill_value=0)
df1_rec_merch_count_mtx #sparse matrix of merchants and users. Values are the number of times a user went to a resturant.

merchant,,and stable,bray gour,brewing compa,hattie b s ho,honey-jam caf,juicyo breakf,lockport stag,merchant,nobel house -,...,z.brew lake zu,zablong,zaxby's,zaza italian,zephyr,ziggi's coffee,zo sushi and thai,zoe's kitchen,zucca bar & pizzeria,zum duernbraeu
uid,,,,,,,,,,,,,,,,,,,,,
005vcYECEUZiN4cXajuea2lBnCy1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0BvDU2CvJTPwox1OYT70bnWdDUB2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0UIAYEH7qlUI6j1iiIg6XbmJWTG3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,63,0,0,0,0
1I87ooK6OBNwxmmA31vkOHwA8wB3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1fPMZehrbgWogwknDB4ZKluoVFg2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,14,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
wrHJZ3ZqRaO9VIPd55VTs6qHOpt2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
xVIvGWUazLftKSenD2ozjaDN2ly2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
yZ8p33MBhcZSwkvUdnPFwnV1RR22,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
merchant_list=df1_rec_merch_count_mtx.columns.tolist()

In [36]:
#df1_rec_merch_meanamt_mtx=df1_rec_resturants.pivot_table(index='uid',columns='merchant', values='amountnum', aggfunc='mean')
#df1_rec_merch_meanamt_mtx #sparse matrix of merchants and users. Values are the number of times a sp user went to a resturant.

In [37]:
#Which merchat has the user just visited? Starting with mcdonald's
resturant_user_visited="mcdonald's"

In [38]:
resturant_popularity_count=df1_rec_merch_count_mtx[resturant_user_visited]#How many times each user visited the "resturant_user_visited"

#resturant_popularity_count[resturant_popularity_count>0] #Get only the rows of user who actually visited the "resturant_user_visited"

## Evaluating Similarity Based on Pearson Correlation

In [39]:
similar_to_resturant = df1_rec_merch_count_mtx.corrwith(resturant_popularity_count)
#similar_to_resturant[similar_to_resturant.isna()]
similar_to_resturant

merchant
                       -0.008459
 and stable             0.349551
 bray gour              0.110878
 brewing compa          0.031320
 hattie b s ho          0.031320
                          ...   
ziggi's coffee          0.660749
zo sushi and thai       0.110878
zoe's kitchen          -0.015112
zucca bar & pizzeria   -0.008459
zum duernbraeu          0.349551
Length: 2225, dtype: float64

In [40]:
corr_resturant = pd.DataFrame(similar_to_resturant, columns=['PearsonR'])
corr_resturant.dropna(inplace=True)
corr_resturant

,PearsonR
merchant,
,-0.008459
and stable,0.349551
bray gour,0.110878
brewing compa,0.031320
hattie b s ho,0.031320
...,...
ziggi's coffee,0.660749
zo sushi and thai,0.110878
zoe's kitchen,-0.015112


In [41]:
step1 = corr_resturant.join(count_per_merch)
resturant_corr_summary=step1.join(meanamt_per_merch)

In [42]:
resturant_corr_summary

,PearsonR,merchant,amountnum
merchant,,,
,-0.008459,1,10.880000
and stable,0.349551,1,6.280000
bray gour,0.110878,1,12.750000
brewing compa,0.031320,4,47.575000
hattie b s ho,0.031320,1,38.090000
...,...,...,...
ziggi's coffee,0.660749,84,7.150119
zo sushi and thai,0.110878,1,28.130000
zoe's kitchen,-0.015112,18,11.765000


In [43]:
resturant_corr_summary.rename(columns={"merchant": "Num_Times_Merch_Was_Visited", 'amountnum':'Mean_Amt_Per_Merch'}, inplace=True)
resturant_corr_summary.sort_values('Num_Times_Merch_Was_Visited', ascending=False).head(10)

,PearsonR,Num_Times_Merch_Was_Visited,Mean_Amt_Per_Merch
merchant,,,
starbucks,0.264495,398,13.753266
dunkin' donuts,0.039006,255,4.649490
chipotle mexican grill,0.300723,255,14.980000
chick-fil-a,0.669982,245,14.616939
firehouse subs,-0.147459,221,9.611946
mcdonald's,1.000000,211,9.725592
christi cafe,0.031320,206,3.382524
sushi aji,-0.178839,171,38.452164
brewing *******et,0.667782,111,11.646216


In [80]:
final_recommendation=resturant_corr_summary[(resturant_corr_summary.PearsonR > 0.7) & (resturant_corr_summary.Num_Times_Merch_Was_Visited>=100) & (resturant_corr_summary.index!= resturant_user_visited) &(resturant_corr_summary.Mean_Amt_Per_Merch < 20)].sort_values('PearsonR', ascending=False).head(5)
final_recommendation.index.tolist()

["wendy's"]

In [45]:
print("Since you visited {}, we recommend you visit {}".format(resturant_user_visited, final_recommendation.index.tolist()))

Since you visited mcdonald's, we recommend you visit ["wendy's"]


In [46]:
#Question for FinGoal: What similarity scores will you be looking for? 

## Evaluating Similarity Based on Cosine Similarity

In [48]:
df1_rec_merch_count_mtx.T

uid,005vcYECEUZiN4cXajuea2lBnCy1,0BvDU2CvJTPwox1OYT70bnWdDUB2,0UIAYEH7qlUI6j1iiIg6XbmJWTG3,1I87ooK6OBNwxmmA31vkOHwA8wB3,1fPMZehrbgWogwknDB4ZKluoVFg2,2TrwNEysErU8rKqlE73HCu1L5i23,4AX2Ah5WTuUxW5CJCiMnkyqnMrs2,5FrXFfLbAmNPPOOdXDWZ4ifaap82,5TZ24fWpnyeVX2yEOSWtEgINSMC2,6KgQR8On4dbB4iOhTmspFs15rC82,...,suZwaVMSKnVmlHdQ4WmbTzgo3E42,tcn1I6zVIaUZtSTnpPvyjRCbkWC2,tkCHZD4yApRvicn92cNXLGGR2yH2,tpgEryFqqtSd5juhC85tim271ut2,vw6j8oVvAfhiLuXYahevZgShRXr2,wrHJZ3ZqRaO9VIPd55VTs6qHOpt2,xVIvGWUazLftKSenD2ozjaDN2ly2,yZ8p33MBhcZSwkvUdnPFwnV1RR22,zMtgoCGe5JeMsQSRfZvKzS3CB9g2,zmpwcQoYK6eCUNN4HoJUUyI5xgs1
merchant,,,,,,,,,,,,,,,,,,,,,
,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
and stable,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
bray gour,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
brewing compa,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
hattie b s ho,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ziggi's coffee,0,0,63,0,14,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
zo sushi and thai,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
zoe's kitchen,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [73]:
temp=cosine_similarity(df1_rec_merch_count_mtx.T)
temp_df=pd.DataFrame(temp, columns=merchant_list, index=merchant_list)
similar_to_resturant_cosine = temp_df[resturant_user_visited]
cosine_resturant=pd.DataFrame(similar_to_resturant_cosine)
cosine_resturant.columns=['CosineSim']
cosine_resturant.dropna(inplace=True)
cosine_resturant

,CosineSim
,0.036102
and stable,0.361020
bray gour,0.144408
brewing compa,0.072204
hattie b s ho,0.072204
...,...
ziggi's coffee,0.654129
zo sushi and thai,0.144408
zoe's kitchen,0.054990
zucca bar & pizzeria,0.036102


In [74]:
step_temp=cosine_resturant.join(count_per_merch)
resturant_cosine_sim_summary=step_temp.join(meanamt_per_merch)
resturant_cosine_sim_summary

,CosineSim,merchant,amountnum
,0.036102,1,10.880000
and stable,0.361020,1,6.280000
bray gour,0.144408,1,12.750000
brewing compa,0.072204,4,47.575000
hattie b s ho,0.072204,1,38.090000
...,...,...,...
ziggi's coffee,0.654129,84,7.150119
zo sushi and thai,0.144408,1,28.130000
zoe's kitchen,0.054990,18,11.765000
zucca bar & pizzeria,0.036102,5,31.046000


In [75]:
resturant_cosine_sim_summary.rename(columns={"merchant": "Num_Times_Merch_Was_Visited", 'amountnum':'Mean_Amt_Per_Merch'}, inplace=True)
resturant_cosine_sim_summary.sort_values('Num_Times_Merch_Was_Visited', ascending=False).head(10)

,CosineSim,Num_Times_Merch_Was_Visited,Mean_Amt_Per_Merch
starbucks,0.425447,398,13.753266
dunkin' donuts,0.088735,255,4.649490
chipotle mexican grill,0.432298,255,14.980000
chick-fil-a,0.711531,245,14.616939
firehouse subs,0.000961,221,9.611946
mcdonald's,1.000000,211,9.725592
christi cafe,0.072204,206,3.382524
sushi aji,0.000000,171,38.452164
brewing *******et,0.649836,111,11.646216
wendy's,0.800645,106,11.853774


In [82]:
final_recommendation=resturant_cosine_sim_summary[(resturant_cosine_sim_summary.CosineSim > 0.7) & (resturant_corr_summary.Num_Times_Merch_Was_Visited>=100) & (resturant_corr_summary.index!= resturant_user_visited) &(resturant_corr_summary.Mean_Amt_Per_Merch < 20)].sort_values('CosineSim', ascending=False).head(5)
final_recommendation.index.tolist()

["wendy's", 'chick-fil-a']

In [83]:
print("Since you visited {}, we recommend you visit {}".format(resturant_user_visited, final_recommendation.index.tolist()))

Since you visited mcdonald's, we recommend you visit ["wendy's", 'chick-fil-a']
